# Introduction

you've learned how to define a NN. it comes with entirely random weights. needs to be trained // needs to be exposed to the data so it can discover patterns

in this lesson, you'll learn big picture about how this works.  then, in the next lesson, you'll learn how to apply these concepts to train a neural network.

you've learned how to define neural networks. have talked at a high level about how they're trained

in this lesson, we go into the details!

# Stochastic gradient descent

how can we minimize the loss? 

idea of loss as a surface.  each different set of parameters is different location on the surface.  want to find parameters that get you to the lowest valley.

how do you do this? start with one set of parameters. get the loss. find the path of steepest descent. take a step. repeat.

will actually do this in the next lesson.  keras takes care of all of this for you

# all the other things too

# Code

... amending parameters

In [ ]:
# especially loss function, batch size, optimizer ...